In [36]:
import os 
import requests
import joblib
import base64 as b64
import pandas as pd

In [38]:
#Create spotify class which contains all functions and attributes related to spotify API
class SpotifyAPI:
    
    authorize_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_authorization_code(self):
        
        client_id = self.client_id
        client_secret = self.client_secret
        authorize_url = self.authorize_url
        client_creds = f'{client_id}:{client_secret}'
        
        client_creds_b64 = b64.b64encode(client_creds.encode())
        
        
        authorize_params = {
            'grant_type': 'client_credentials'
        }
        
        token_header = {
            'Authorization': f'Basic {client_creds_b64.decode()}' 
        }
        
        r = requests.post(authorize_url, data=authorize_params, headers=token_header)
        r_json = r.json()
        
        access_token = r_json['access_token']
        
        return access_token

In [39]:
#Create spotify client object with spotify app client id and secret key
spotify_client_id = os.environ['SPOTIFY_API_CLIENT_ID']
spotify_secret_key = os.environ['SPOTIFY_API_SECRET_KEY']

spotify_client = SpotifyAPI(spotify_client_id, spotify_secret_key)

In [40]:
#Get access token
access_token = spotify_client.get_authorization_code()

In [41]:
#Get pollen playlist spotify track ids
pollen_playlist_id = '37i9dQZF1DWWBHeXOYZf74'

pollen_playlist_tracks = []

playlist_header = {
    'Authorization': f'Bearer {access_token}'
}

for i in range(5):
    
    playlist_response = requests.get(f'https://api.spotify.com/v1/playlists/{pollen_playlist_id}/tracks?offset={i*100}', headers=playlist_header)
    playlist_response_json = playlist_response.json()
    
    for item in playlist_response_json['items']:
        
        track_id = item['track']['id']
        pollen_playlist_tracks.append(track_id)

In [42]:
#Load liked tracks ids
liked_track_ids = joblib.load('../Joblib_Objects/liked_track_ids')

#Filter playlist for songs that are already liked
filtered_pollen_playlist_tracks = []

for track in pollen_playlist_tracks:
    if track not in liked_track_ids:
        filtered_pollen_playlist_tracks.append(track)

In [43]:
#Get track information for all filtered pollen playlist tracks
pollen_playlist_df = pd.DataFrame(columns=['genre','danceability','energy','key','loudness','mode','speechiness','acousticness',
                                         'instrumentalness','liveness','valence','tempo'])

track_header = {
    'Authorization': f'Bearer {access_token}'
}

for track in filtered_pollen_playlist_tracks:
    try:
        audio_features_response = requests.get(f'https://api.spotify.com/v1/audio-features/{track}', headers=track_header)
        audio_features_response_json = audio_features_response.json()

        track_response = requests.get(f'https://api.spotify.com/v1/tracks/{track}',headers=track_header)
        track_response_json = track_response.json()
        
        artist_id = track_response_json['artists'][0]['id']
        
        artist_response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}',headers=track_header)
        artist_response_json = artist_response.json()
        genres = artist_response_json['genres']
        
        track_features = {
            'genre':genres,
            'danceability':audio_features_response_json['danceability'],
            'energy':audio_features_response_json['energy'],
            'key':audio_features_response_json['key'],
            'loudness':audio_features_response_json['loudness'],
            'mode':audio_features_response_json['mode'],
            'speechiness':audio_features_response_json['speechiness'],
            'acousticness':audio_features_response_json['acousticness'],
            'instrumentalness':audio_features_response_json['instrumentalness'],
            'liveness':audio_features_response_json['liveness'],
            'valence':audio_features_response_json['valence'],
            'tempo':audio_features_response_json['tempo']
        }

        pollen_playlist_df = pollen_playlist_df.append(track_features,ignore_index=True)
        
    except:
        
        print(track)

In [44]:
#Format genres to only have 1 listed
def replace_empty_list(track):
    if len(track) == 0:
        track = ['Not Available']
    else:
        pass
    
    return track

pollen_playlist_df['genre'] = pollen_playlist_df['genre'].map(replace_empty_list)
pollen_playlist_df['genre'] = pollen_playlist_df['genre'].map(lambda x: x[0])

In [45]:
#load label encoder
genres_label_encoder = joblib.load('../Joblib_Objects/genres_label_encoder')

def label_encode(track):
    
    try:
        encoded_genre = genres_label_encoder.transform([track])[0]
        
        return int(encoded_genre)
    
    except:
        pass
    
pollen_playlist_df['genre'] = pollen_playlist_df['genre'].map(int(label_encode))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'function'

In [ ]:
#Drop rows with nans
pollen_playlist_df = pollen_playlist_df.dropna()

In [ ]:
#Load scaler
scaler = joblib.load('../Joblib_Objects/scaler')
X = scaler.transform(pollen_playlist_df)

In [ ]:
#Load model
xgb_model = joblib.load('../spotify_song_classifier.dat')
pollen_playlist_df